In [1]:
import numpy as np
import pandas as pd
import altair as alt
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore', ValueWarning)

import fidap
import config

# instantiate connection
fidap = fidap.fidap_client(api_key = config.api_key)

### FRED  and Housing Data  
  
**Metadata Querying**
  
The Federal Reserve Bank of St. Louis uploads a ton of useful economic data that we can access using `fidap`. Suppose we know that this FRED dataset resides in the backend of GoogleBigQuery but it is not visble in the front end. That means that we cannot easily determine its schema. What can we do? What are some of the GoogleBigQuery SQL functions that we can run to help us?    

In [ ]:
fred_meta = fidap.sql("""
SELECT *
FROM fidap-301014.fred.INFORMATION_SCHEMA.TABLES
""")

# get all table names
print(fred_meta.table_name)

#meta for each of the tables to figure out which might be useful 
categories_meta = fidap.sql("""
SELECT * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM fidap-301014.fred.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'categories'
""")

series_info_meta = fidap.sql("""
SELECT * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM fidap-301014.fred.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'series_info'
""")

series_observations_meta = fidap.sql("""
SELECT * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM fidap-301014.fred.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'series_observations'
""")

# id the series that should be available in FRED project
available_series = fidap.sql("""
WITH all_series AS (
SELECT DISTINCT series_id 
FROM fidap-301014.fred.series_observations
)

SELECT * 
FROM fidap-301014.fred.series_info AS i
INNER JOIN all_series as s
ON s.series_id = i.series_id;
""")

**Initial Housing Data Extraction**  
  
Suppose I want information on the following tables that can provide us with more information on the housing market in US:  
  
1) Monthly Supply of Houses in the United States  
2) New Privately-Owned Housing Units Started: Total Units  
3) New Houses Sold by Stage of Construction, Total  
4) New One Family Houses for Sale in the United States  
5) Producer Price Index by Commodity: Lumber and Wood Products: Hardwood Lumber  
6) Producer Price Index by Commodity: Lumber and Wood Products: Oak and Maple Hardwood Flooring    
7) New Privately-Owned Housing Units Authorized in Permit-Issuing Places: Total Units  
8) Retail Sales of Building Materials, Furnishings and Appliances
  
I can obtain some information from the `series_info` table of the `fred` project.  

In [ ]:
housing_data = fidap.sql("""
SELECT *
FROM fidap-301014.fred.series_info
WHERE series_id IN ('HOUST', 'MSACSR', 'NHSDPTS', 'HNFSUSNSA', 'WPU0812', 'WPU08120401', 'COMPUTSA', 'UNDCONTSA', 'RELDCBMO27SBOG', 'H8B1026NCBCMG', 'MRTSSM444USN', 'MRTSSM4423XUSS', 'PRRESCONS', 'MPCV00XXS', 'PERMIT')
""")

# series that are not available irl ('MSACSR','WPU080120401', 'NHSDPTS')

The interesting thing here is that FRED has stored its data in true tidy data format. Observations from all series are in one table with just a few columns. To speed the querying process, we are going to query for all of the series that we want, for now, in one shot. Then we can filter and extract them accordingly locally.   

In [2]:
# querying for all data

all_housing_data = fidap.sql("""
SELECT series_id, date, value
FROM fidap-301014.fred.series_observations
WHERE series_id IN ('HOUST', 'HOUSTNSA','WPU0812', 'H8B1026NCBCMG', 'MRTSSM444USS', 'MRTSSM4423XUSS', 'MRTSSM4423XUSN', 'MRTSSM444USN', 'PRRESCONS', 'MPCV00XXS')
""")

In [8]:
# housing construction has started
housing_started = all_housing_data[(all_housing_data['series_id'] == 'HOUST')].copy()
housing_started_nsa = all_housing_data[(all_housing_data['series_id'] == 'HOUSTNSA')].copy()

# lumber pricing index
hardwood_lumber_pricing = all_housing_data[(all_housing_data['series_id'] == 'WPU0812')].copy()

# real estate loans 
delta_real_estate_loans = all_housing_data[(all_housing_data['series_id'] == 'H8B1026NCBCMG')].copy()

# private residential spending
building_materials_expenditure = all_housing_data[(all_housing_data['series_id'] == 'MRTSSM444USS')].copy()
furnishing_expenditure = all_housing_data[(all_housing_data['series_id'] == 'MRTSSM4423XUSS')].copy()

# housing expenditure
private_residential_expenditure = all_housing_data[(all_housing_data['series_id'] == 'PRRESCONS')].copy()
residential_construction_expenditure = all_housing_data[(all_housing_data['series_id'] == 'MPCV00XXS')].copy()

**Housing-Related Retail Sales: Regression and Seasonal Patterns**

What I would really like to find out is whether we can eyeball or identify any relationship between supply of houses and construction of new houses. A quick inspection of the data tells us that these series do not all start at the same time.   
  
Let us start by seeing if there is a relationship between retail spending of building materials and home furnishings.  

In [4]:
# joining the two datasets
building_furnishing = furnishing_expenditure.merge(building_materials_expenditure, on = 'date', how = 'inner')

# building the plot
furnishing_scatter_base = alt.Chart(building_furnishing,
                                    title = "Building Materials and Home Furnishings Retail Sales").mark_circle().encode(
    x = alt.X('value_y',
              title = 'Building Materials (Mil. of $)'),
    y = alt.Y('value_x',
              title = 'Home Furnishings and Appliances (Mil. of $)'))

furnishing_poly_fit = [
    furnishing_scatter_base.transform_regression(
        'value_y', 'value_x', method = 'poly', order = 3
    ).mark_line(color = '#fb8072').encode()
]

alt.layer(furnishing_scatter_base, furnishing_scatter_base.transform_regression('value_y', 'value_x', method = 'log').mark_line(color = '#fdc086'), *furnishing_poly_fit)

alt.LayerChart(...)

`Altair` lets us run and plot regressions directly. But they are rather simple ones. We can start by modelling the relationship between expenditure on building materials and home furnishings as a third-order polynomial:  
  
${Home Furnishings} = {a(Building Materials)}^{3} + {b(Building Materials)}^{2} + {c(Building Materials)} + {d} $  
  
This is represented by the red line. But we also know that the above regression function is not quite it. There is a great deal of error at the higher end.    
  
We can try another regression function. The orange is a logarithmic regression function fitted to the distribution of the data. This can be expressed as:    
  
${Home Furnishings} = {a} + {log(Building Materials)}^{b}$  

It is often said that peak building season is during the warmer summer months. Can we see this in the data? 

In [6]:
# getting unadjusted data
home_furnishing_building_materials_nsa = all_housing_data[(all_housing_data['series_id'] == 'MRTSSM4423XUSN')|(all_housing_data['series_id'] == 'MRTSSM444USN')].copy()

# getting month
home_furnishing_building_materials_nsa['month'] = home_furnishing_building_materials_nsa['date'].dt.month

# descriptive titles
id_title_map = {
    'MRTSSM4423XUSN': 'Home Furnishing and Appliances',
    'MRTSSM444USN': 'Home Building Materials'
}
home_furnishing_building_materials_nsa['series'] = home_furnishing_building_materials_nsa['series_id'].map(id_title_map)

# building the plot
home_furnishing_building_nsa_scatter = alt.Chart(home_furnishing_building_materials_nsa).mark_circle(
    opacity = 0.4).encode(
    x = alt.X('month', title = 'Month'),
    y = alt.Y('value', title = 'Retail Sales (Mil of $)'),
    color = 'series'
).properties(
    title = {
        "text": "Retail Sales of Furniture and Building Materials",
        "subtitle": "Not Seasonally Adjusted"
    })

home_furnishing_building_nsa_scatter.configure_title(anchor = 'start')
    
alt.layer(home_furnishing_building_nsa_scatter, home_furnishing_building_nsa_scatter.transform_loess('month', 'value', groupby = ['series']).mark_line())

alt.LayerChart(...)

Interestingly enough, we do not see the same seasonal spike in the summer months for home furnishing and appliances. Instead, the spike occurs at the end of the year. This is probably due to the Thanksgiving and the Christmas holiday season.  
  
**Housing Units Construction: Seasonal Adjustment and the concept of Stationarity**  
  
What about actual housing construction? We can look at it in terms of the number of units where work has just started in each month.  

In [9]:
# extract month from date column 
housing_started['month'] = housing_started['date'].dt.month
housing_started_nsa['month'] = housing_started_nsa['date'].dt.month

In [10]:
housing_started_nsa_chart = alt.Chart(housing_started_nsa).mark_circle(
    opacity = 0.4,
    color = '#fdb462'
).encode(
    x = alt.X(
        'month',
        title = 'Month'),
    y = alt.Y(
        'value',
        title = 'Thousands of Units')
).properties(
    title = {
        "text": "New Housing Units Started",
        "subtitle": "Not Seasonally Adjusted"
    }
)

housing_started_nsa_chart.configure_title(
    anchor = 'start'
)

alt.layer(housing_started_nsa_chart, housing_started_nsa_chart.transform_loess('month', 'value').mark_line(color = '#8dd3c7'))

alt.LayerChart(...)

Cool. Suppose we want to do some sort of time series modelling and forecasting on this dataset. Most time series modelling requires a dataset to be stationary. A dataset that arises from a stationary process is one whose parameters such as mean and variance does not change over time, to put it simply. That is, th data-generating process does not experience seasonal differences.  
  
We already know that housing construction is highly seasonal. But how do we prove it? We can conduct a hypothesis test using the `Augmented Dickey-Fuller (ADF)` test.   

In [12]:
housing_values = housing_started_nsa.loc[:,'value'].values
adf_result = adfuller(housing_values, autolag = 'AIC')

print(f'ADF Statistic: {adf_result[0]}')
print(f'n_lags: {adf_result[1]}')
print(f'p-value: {adf_result[1]}')
for key, value in adf_result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}') 

ADF Statistic: -4.062971069000669
n_lags: 0.0011143974398532632
p-value: 0.0011143974398532632
Critial Values:
   1%, -3.439314999916068
Critial Values:
   5%, -2.8654965012008677
Critial Values:
   10%, -2.5688768817372867


This is very interesting because even though we know that the data is seasonal, the ADF statistic is lower than the critical values, and the p-value is less than 0.01. There is no reason to reject the null hypothesis that the data-generating process is stationary. But we must remember that there could be broader factors affecting the rate of housing construction such as macroeconomic cycle etc that causes big swings from year to year. So what if we just look at the first five years of data? 

In [23]:
housing_values = housing_started_nsa.loc[:,'value'].values
housing_values = housing_values[0:60]
adf_result = adfuller(housing_values)

print(f'ADF Statistic: {adf_result[0]}')
print(f'n_lags: {adf_result[1]}')
print(f'p-value: {adf_result[1]}')
for key, value in adf_result[4].items():
    print('Critial Values:')
    print(f'   {key}, {value}')

ADF Statistic: -0.26105905956386594
n_lags: 0.930841326288828
p-value: 0.930841326288828
Critial Values:
   1%, -3.5714715250448363
Critial Values:
   5%, -2.922629480573571
Critial Values:
   10%, -2.5993358475635153


In this case, yes, the ADF statistic is higher than any of the critical values, and the p-value is more than 0.01. In other words, when subset into smaller chunks, the dataset is non-stationary. In fact, we can query the seasonally adjusted chart from FRED, plot it, and see how different it is from the non-adjusted chart. 

In [6]:
housing_started_chart = alt.Chart(housing_started).mark_circle(
    opacity = 0.4,
    color = '#fdb462'
).encode(
    x = alt.X(
        'month',
        title = 'Month'),
    y = alt.Y(
        'value',
        title = 'Thousands of Units')
).properties(
    title = {
        "text": "New Housing Units Started",
        "subtitle": "Seasonally Adjusted"
    }
)

housing_started_chart.configure_title(
    anchor = 'start'
)
alt.layer(housing_started_chart, housing_started_chart.transform_loess('month', 'value').mark_line(color = '#8dd3c7'))

alt.LayerChart(...)

To prove my point, let us look at it over the first 30 years of record keeping. 

In [24]:
housing_started_nsa_59_89 = housing_started_nsa.iloc[:420,].copy()

housing_started_nsa_year_chart = alt.Chart(housing_started_nsa_59_89).mark_line(
    opacity = 0.8,
    color = '#fdb462'
).encode(
    x = alt.X(
        'date',
        title = 'Years'),
    y = alt.Y(
        'value',
        title = 'Thousands of Units')
).properties(
    title = {
        "text": "New Housing Units Started",
        "subtitle": "Not Seasonally Adjusted"
    }
).interactive()

housing_started_nsa_year_chart.configure_title(
    anchor = 'start'
)

alt.Chart(...)

Roughly speaking, yes there is some pattern and seasonality. This means that the data is non-stationary. Can we try to do some predicting using ARIMA? 

In [37]:
# non-seasonal model
non_seasonal_model = ARIMA(housing_started_nsa_59_89['value'], order = (1,1,1))
ns_results = non_seasonal_model.fit()
housing_started_nsa_59_89['non_seasonal_forecast_value'] = ns_results.predict(start = 408, dynamic = True)

# seasonal model
seasonal_model = sm.tsa.statespace.SARIMAX(housing_started_nsa_59_89['value'], 
                                          order = (1,1,1),
                                          seasonal_order = (1,1,1,12))
results = seasonal_model.fit()
housing_started_nsa_59_89['seasonal_forecast_value'] = results.predict(start = 408, dynamic = True)

housing_started_nsa_year_prediction_chart = alt.Chart(housing_started_nsa_59_89).mark_line(
    opacity = 0.8,
    color = '#b3de69'
).encode(
    x = alt.X(
        'date',
        title = 'Years'),
    y = alt.Y(
        'seasonal_forecast_value',
        title = 'Thousands of Units')
).interactive()

housing_started_nsa_year_ns_prediction_chart = alt.Chart(housing_started_nsa_59_89).mark_line(
    opacity = 0.8,
    color = '#1f78b4'
).encode(
    x = alt.X(
        'date',
        title = 'Years'),
    y = alt.Y(
        'non_seasonal_forecast_value',
        title = 'Thousands of Units')
).interactive()

alt.layer(housing_started_nsa_year_chart, housing_started_nsa_year_prediction_chart, housing_started_nsa_year_ns_prediction_chart)

alt.LayerChart(...)

The blue line is the 12-month forecast that assumes the data to be stationary (i.e. non-seasonal), and the green is a forecast that assumes the data is non-stationary and seasonal. We can clearly see that seasonality is fairly real here as the seasonal forecast tracks the actual number of units where construction start a lot more closely.

From a more meta perspective, can we look at how much total private expenditure on residential construction has changed over the years? 

In [29]:
alt.Chart(
    residential_construction_expenditure,
    title = 'Total Private Residential Construction Expenditure').mark_bar().encode(
    x = alt.X(
        'date',
        title = "Year"),
    y = alt.Y(
        'value',
        title = "% Change From Preceeding Period"),
    color = alt.condition(
        alt.datum.value > 0,
        alt.value('#ccebc5'),
        alt.value('#fbb4ae')),
    tooltip = ['date', 'value']
)

alt.Chart(...)

What I find interesting is that the expenditure dipped for a very short period of time in 2020 even amidst a pandemic-induced recession. The decline, while sharp, was also very short in comparison to the period right before and during the 2008 recession. Are the changes in total private residential construction expenditure correlated with the gap between construction permits issued and the number of housing units where work has started?   

**Permitting and Construction: Time Lags**

In [30]:
# by running this query,
# we elim. the step of having to figure out when the series started or ended
# we can also calculate the permit-construction commencement gap directly

construction_permit_gap = fidap.sql("""
WITH permit AS ( 
SELECT date, CAST(value AS NUMERIC) AS permits,
(LAG(CAST(value AS NUMERIC), 1) OVER (ORDER BY date ASC)) AS permits_lagged_1,
(LAG(CAST(value AS NUMERIC), 2) OVER (ORDER BY date ASC)) AS permits_lagged_2
FROM fidap-301014.fred.series_observations
WHERE series_id = 'PERMIT'),

gap AS(
SELECT c.date, 
CAST(p.permits AS FLOAT64)-CAST(c.value AS NUMERIC) AS gap,
p.permits_lagged_1 - CAST(c.value AS NUMERIC) AS gap_lagged_1,
p.permits_lagged_2 - CAST(c.value AS NUMERIC) AS gap_lagged_2
FROM fidap-301014.fred.series_observations AS c
INNER JOIN permit AS p
ON p.date = c.date
WHERE series_id = 'HOUST')

SELECT e.date, e.value AS ex_delta, g.gap, g.gap_lagged_1, g.gap_lagged_2
FROM fidap-301014.fred.series_observations AS e
INNER JOIN gap AS g
ON g.date = e.date
WHERE series_id = 'MPCV00XXS';
""")

In [31]:
gap_ex_scatter = alt.Chart(construction_permit_gap).mark_circle(color = '#e5c494').encode(
    x = alt.X('ex_delta', title = "Change in Residential Construction Expenditure (%)"),
    y = alt.Y('gap', title = 'Permits and Construction Gap (Thousand Units)')
).properties(
    title = {
        "text": "Construction, Permitting, and Expenditure",
        "subtitle": "No Time Lag Betw. Permit Issued and Construction Commencement"
    })

alt.layer(gap_ex_scatter, gap_ex_scatter.transform_regression('ex_delta', 'gap', method = 'linear').mark_line(color = '#bebada'))

alt.LayerChart(...)

Interestingly enough, this is not what we expected. We would expect periods of increased residential construction expenditure to correlate with months where commencement of construction outstrips permits issued.  
  
What if we introduce a lag? There could be a time lag between the issuance of a permit and the commencement of construction.  

In [32]:
gap_ex_lag1_scatter = alt.Chart(construction_permit_gap).mark_circle(color = '#e5c494').encode(
    x = alt.X('ex_delta', title = "Change in Residential Construction Expenditure (%)"),
    y = alt.Y('gap_lagged_1', title = 'Permits and Construction Gap (Thousand Units)')
).properties(
    title = {
        "text": "Construction, Permitting, and Expenditure",
        "subtitle": "One Month's Time Lag Betw. Permit Issued and Construction Commencement"
    })

alt.layer(gap_ex_lag1_scatter, gap_ex_lag1_scatter.transform_regression('ex_delta', 'gap_lagged_1', method = 'linear').mark_line(color = '#bebada'))

alt.LayerChart(...)

In [33]:
gap_ex_lag2_scatter = alt.Chart(construction_permit_gap).mark_circle(color = '#e5c494').encode(
    x = alt.X('ex_delta', title = "Change in Residential Construction Expenditure (%)"),
    y = alt.Y('gap_lagged_2', title = 'Permits and Construction Gap (Thousand Units)')
).properties(
    title = {
        "text": "Construction, Permitting, and Expenditure",
        "subtitle": "Two Months' Time Lag Betw. Permit Issued and Construction Commencement"
    })

alt.layer(gap_ex_lag2_scatter, gap_ex_lag2_scatter.transform_regression('ex_delta', 'gap_lagged_2', method = 'linear').mark_line(color = '#bebada'))

alt.LayerChart(...)

Predictably enough, the relationship starts to show once a time lag is introduced. Hence, we can say that as residential construction expenditure increases, month on month, the deficit between the number of construction permits issued and construction commencement grows.